In [51]:
import pandas as pd

all_skills = 'skills/Skills.xlsx'
digital_skills = 'skills/Digital Skills.xlsx'
coursera_skills = 'coursera_skills/Coursera Skills To DB.xlsx'

In [52]:
def extract_skills(file_name):
    xl_file = pd.ExcelFile(file_name)
    dfs = {sheet_name: xl_file.parse(sheet_name) for sheet_name in xl_file.sheet_names}
    all_skills = []
    for key, df in dfs.items():
        all_skills.extend(df['Skill'].unique().tolist())
    df_skills = pd.DataFrame({'skill': all_skills})
    return df_skills.drop_duplicates()

def parent_check(file_name, df_skills):
    xl_file = pd.ExcelFile(file_name)
    dfs = {sheet_name: xl_file.parse(sheet_name) for sheet_name in xl_file.sheet_names}
    for key, df in dfs.items():
        df_error = df.loc[~df['Parent'].isin(df_skills['skill'])].dropna()
        print(df_error['Parent'].unique().tolist())

def extract_relation(file_name, df_skills):
    xl_file = pd.ExcelFile(file_name)
    dfs = {sheet_name: xl_file.parse(sheet_name) for sheet_name in xl_file.sheet_names}
    df_rel = pd.DataFrame(columns=['Skill', 'Parent'])
    skill_to_id_dict = df_skills.set_index('skill').to_dict()['skill_id']
    for key, df in dfs.items():
        df_rel = df_rel.append(df.replace(skill_to_id_dict))
    df_rel = df_rel.reset_index(drop=True)
    df_rel = df_rel.dropna()
    df_rel = df_rel.drop_duplicates()
    df_rel['Parent'] = df_rel['Parent'].astype(int)
    return df_rel

### Extract Skills

In [53]:
# All skills
df_s1 = extract_skills(all_skills)
df_s1.head()

,skill
0,Applied Science
1,Arts
2,Business
3,Computer Science
4,Data Science


In [54]:
df_s1.describe(include='object')

,skill
count,1781
unique,1781
top,Parsing
freq,1


In [55]:
# Digital skills
df_s2 = extract_skills(digital_skills)
df_s2.head()

,skill
0,Data Analysis
1,Machine Learning
2,Algorithm
3,Artificial Intelligence
4,Big Data


In [56]:
df_s2.describe(include='object')

,skill
count,984
unique,984
top,Public Key Cryptography Standards (PKCS)
freq,1


In [57]:
# Combine
df_skills = df_s1.append(df_s2)
df_skills = df_skills.drop_duplicates()
df_skills = df_skills.dropna()
df_skills.describe()

,skill
count,2588
unique,2588
top,Web Application Development
freq,1


### Coursera Skills

In [58]:
df_c = pd.read_excel(coursera_skills)
df_c.head()

,skill_id,c_skill,Skill,Parent
0,457,Low Poly,Low Poly,3D Computer Graphics
1,462,Polygon Mesh,Polygon Mesh,3D Computer Graphics
2,466,Uv Mapping,Uv Mapping,3D Computer Graphics
3,467,Framing,Framing,3D Computer Graphics
4,469,3D Graphics Software,3D Graphics Software,3D Computer Graphics


In [59]:
df_coursera = df_c[['skill_id', 'c_skill']]
df_coursera = df_coursera.rename(columns={'c_skill': 'skill'})
df_coursera.head()

,skill_id,skill
0,457,Low Poly
1,462,Polygon Mesh
2,466,Uv Mapping
3,467,Framing
4,469,3D Graphics Software


In [60]:
df_others = df_c[['Skill']].rename(columns={'Skill': 'skill'}).dropna()
# Combine
df_skills = df_skills.append(df_others)
df_skills = df_skills.drop_duplicates()
df_skills['skill_id'] = df_skills.groupby('skill').grouper.group_info[0]
df_skills = df_skills.reset_index(drop=True)
df_skills = df_skills[['skill_id', 'skill']]
df_skills.head()

,skill_id,skill
0,415,Applied Science
1,473,Arts
2,988,Business
3,1634,Computer Science
4,2099,Data Science


In [61]:
df_skills.describe(include='all')

,skill_id,skill
count,9134.000000,9134
unique,NaN,9134
top,NaN,Boy
freq,NaN,1
mean,4566.500000,NaN
std,2636.903013,NaN
min,0.000000,NaN
25%,2283.250000,NaN
50%,4566.500000,NaN
75%,6849.750000,NaN


### Data Checking/Cleaning

In [62]:
parent_check(all_skills, df_skills)

[]
[]
[]
[]
[]
[]
[]


In [63]:
parent_check(digital_skills, df_skills)

[]
[]
[]
[]
[]
[]


In [64]:
parent_check(coursera_skills, df_skills)

[]


### Extract Relation

In [65]:
df_r1 = extract_relation(all_skills, df_skills)
df_r2 = extract_relation(digital_skills, df_skills)
# Combine
df_rel = df_r1.append(df_r2)
df_rel = df_rel.drop_duplicates()
df_rel = df_rel.reset_index(drop=True)
df_rel = df_rel.rename(columns={'Skill': 'skill_1_id', 'Parent': 'skill_2_id'})
df_rel.head()

,skill_1_id,skill_2_id
0,4746,5593
1,6315,5593
2,8389,1634
3,8389,5096
4,76,988


In [66]:
df_rel.describe(include='all')

,skill_1_id,skill_2_id
count,3079.0,3079.000000
unique,2579.0,NaN
top,7266.0,NaN
freq,7.0,NaN
mean,NaN,4400.757389
std,NaN,2630.121067
min,NaN,4.000000
25%,NaN,2091.500000
50%,NaN,4660.000000
75%,NaN,6665.000000


In [85]:
df_crel = df_c[['skill_id', 'Skill']].rename(columns={'skill_id': 'coursera_skill_id', 'Skill': 'skill'})\
    .merge(df_skills, how='left').drop(columns=['skill'])
df_crel.head()

,coursera_skill_id,skill_id
0,457,4894
1,462,6397
2,466,8683
3,467,3338
4,469,15


In [86]:
df_crel.describe(include='all')

,coursera_skill_id,skill_id
count,7301.000000,7301.000000
mean,3650.941652,4526.222846
std,2107.856673,2625.726614
min,0.000000,0.000000
25%,1826.000000,2254.000000
50%,3651.000000,4512.000000
75%,5476.000000,6791.000000
max,7301.000000,9133.000000


### Export

In [87]:
df_skills.to_csv('skills_db/skill.csv', index=False)
df_rel.to_csv('skills_db/skill_tree.csv', index=False)
df_coursera.to_csv('skills_db/coursera_skill.csv', index=False)
df_crel.to_csv('skills_db/coursera_map.csv', index=False)